# Prompt Validation for Large Language Models (LLMs)

This notebook guides you through using a large language model (LLM) specialized in sentiment analysis for financial news. It shows you how to set up the ValidMind Developer Framework, initializes the client library, and uses a specific prompt template for analyzing the sentiment of given sentences. The notebook also includes example data to test the model's ability to correctly identify sentiment as positive, negative, or neutral.

## ValidMind at a glance

We offer a platform for managing model risk, including risk associated with AI and statistical models. As a model developer, you use the _ValidMind Developer Framework_ to automate documentation and validation tests, and then use the _ValidMind AI Risk Platform UI_ to collaborate on doucumentation projects. Together, these products simplify model risk management, facilitate compliance with regulations and institutional standards, and enhance collaboration between yourself and model validators.

If this is your first time trying out ValidMind:

- [Get started](https://docs.validmind.ai/guide/get-started.html) — The basics, including key concepts, and how our products work
- [Get started with the ValidMind Developer Framework](https://docs.validmind.ai/guide/get-started-developer-framework.html) —  The path for developers, more code samples, and our developer reference

## Before you begin

To use the ValidMind Developer Framework with a Jupyter notebook, you need to install and initialize the client library first, along with getting your Python environment ready. This includes installing any missing prerequisite modules that you discover with `pip install`. 

If you don't already have one, [create a documentation project](https://docs.validmind.ai/guide/create-your-first-documentation-project.html) for yourself in the Platform UI. You will use this project to upload your documentation and test results.

## Install the client library

In [1]:
%pip install -q validmind

Note: you may need to restart the kernel to use updated packages.


## Initialize the client library

Every documentation project in the Platform UI comes with a _code snippet_ that lets the client library associate your documentation and tests with the right project on the Platform UI when you run this notebook.

First, locate your code snippet: 

1. In a browser, log into the [Platform UI](https://app.prod.validmind.ai).

2. Go to **Documentation Projects** > **YOUR_DOCUMENTATION_PROJECT** > **Getting Started** and click **Copy snippet to clipboard**.
   
   ::: {.callout-tip}
   
   This step requires a documentation project. [Learn how you can create one](https://docs.validmind.ai/guide/create-your-first-documentation-project.html).

   :::

Next, replace this placeholder with your own code snippet:

In [ ]:
## Replace this placeholder with the code snippet from your project ## 

import validmind as vm

vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  api_key = "...",
  api_secret = "...",
  project = "..."
)

### Preview the template

A template predefines sections for your documentation project and provides a general outline to follow, making the documentation process much easier.

You will upload documentation and test results into this template later on. For now, take a look at the structure that the template provides with the `vm.preview_template()` function from the ValidMind library and note the empty sections:

In [ ]:
vm.preview_template()

### Download the test dataset
https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news
Download the dataset in the above link and move it into the current directory.

In [ ]:
from validmind.models import FoundationModel, Prompt

In [ ]:
import os

import dotenv
dotenv.load_dotenv()

if os.getenv("OPENAI_API_KEY") is None:
    raise Exception("OPENAI_API_KEY not found")

In [ ]:
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

def call_model(prompt):
    return openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ]
    ).choices[0].message["content"]

In [ ]:
prompt_template = """
You are an AI with expertise in sentiment analysis, particularly in the context of financial news.
Your task is to analyze the sentiment of a specific sentence provided below.
Before proceeding, take a moment to understand the context and nuances of the financial terminology used in the sentence.

Sentence to Analyze:
```
{Sentence}
```

Please respond with the sentiment of the sentence denoted by one of either 'positive', 'negative', or 'neutral'.
Please respond only with the sentiment enum value. Do not include any other text in your response.

Note: Ensure that your analysis is based on the content of the sentence and not on external information or assumptions.
""".strip()

prompt_variables = ["Sentence"]

In [ ]:
import pandas as pd

df = pd.read_csv('./datasets/sentiments.csv')

df_test = df[:10].reset_index(drop=True)
df_test

In [ ]:
vm_test_ds = vm.init_dataset(
    dataset=df_test,
    text_column="Sentence",
    target_column="Sentiment",
)

vm_model = FoundationModel(
    predict_fn=call_model,
    prompt=Prompt(
        template=prompt_template,
        variables=prompt_variables,
    ),
    test_ds=vm_test_ds,
)

In [ ]:
# Run the full suite with the following command (it will take a while):
#
# suite_results = vm.run_documentation_tests(dataset=vm_test_ds, model=vm_model)
#
# By default the tests will use GPT-3.5 as the evaluation model, to get better results use GPT:
# os.environ["VM_OPENAI_MODEL"] = "gpt4"

test_plan_results = vm.run_test_plan("prompt_validation", model=vm_model)